# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [3]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Conv2D, Convolution2D, Permute, BatchNormalization, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [4]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [6]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
print(K.image_data_format())
if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

channels_last
Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 20, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
____________________________________

2. Implementación de la solución DQN

In [8]:
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)

processor = AtariProcessor()

In [9]:
policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=1.,
    value_min=.1,
    value_test=.2,
    nb_steps=100000
)

In [10]:

dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup=200000,   # Aumento el número de pasos iniciales porque el entorno es complejo y así obtiene una base sólida de experiencias
    gamma=.99,
    target_model_update=10000,  # Mantengo las mismas actualizaciones de las redes. Valores altos suponen mayor estabilidad pero más tiempo de entrenamiento
    train_interval=4,
    enable_dueling_network=True,
    dueling_type='avg'
)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

In [8]:
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=500)]

# Loading the weights from the last checkpoint before starting training
#dqn.load_weights('dqn_SpaceInvaders-v0_weights_1200000.h5f')

dqn.fit(env, callbacks=callbacks, nb_steps=1000000, log_interval=1000, visualize=False, verbose=2)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1000000 steps ...


c:\Users\Usuario\anaconda3\envs\ar\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    420/1000000: episode: 1, duration: 11.294s, episode steps: 420, steps per second:  37, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.481 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    960/1000000: episode: 2, duration: 2.581s, episode steps: 540, steps per second: 209, episode reward:  6.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.398 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1356/1000000: episode: 3, duration: 1.816s, episode steps: 396, steps per second: 218, episode reward:  6.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.515 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1855/1000000: episode: 4, duration: 2.357s, episode steps: 499, steps per second: 212, episode reward:  3.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 2.397 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2301/1000000: episode: 5, duration: 2.054s, episode steps: 446, step

c:\Users\Usuario\anaconda3\envs\ar\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 200609/1000000: episode: 255, duration: 24.674s, episode steps: 1486, steps per second:  60, episode reward: 16.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 3.061 [0.000, 5.000],  loss: 2.339366, mae: 1.327838, mean_q: 1.682809, mean_eps: 0.100000
 201552/1000000: episode: 256, duration: 27.208s, episode steps: 943, steps per second:  35, episode reward: 14.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 1.470 [0.000, 5.000],  loss: 0.939694, mae: 0.999743, mean_q: 1.357863, mean_eps: 0.100000
 202868/1000000: episode: 257, duration: 38.303s, episode steps: 1316, steps per second:  34, episode reward: 16.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.166 [0.000, 5.000],  loss: 0.440782, mae: 0.770223, mean_q: 1.128582, mean_eps: 0.100000
 203710/1000000: episode: 258, duration: 24.588s, episode steps: 842, steps per second:  34, episode reward:  8.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.224 [0.000, 5.000],  loss: 0.253658, mae: 0.640712,

In [11]:
weights_filename="dqn_SpaceInvaders-v0_weights.h5f"
dqn.load_weights(weights_filename)
test=dqn.test(env, nb_episodes=10, visualize=True)
print("La media del reward es:", np.mean(test.history["episode_reward"]))

Testing for 10 episodes ...


c:\Users\Usuario\anaconda3\envs\ar\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 17.000, steps: 976
Episode 2: reward: 17.000, steps: 976
Episode 3: reward: 17.000, steps: 963
Episode 4: reward: 17.000, steps: 963
Episode 5: reward: 17.000, steps: 962
Episode 6: reward: 17.000, steps: 973
Episode 7: reward: 17.000, steps: 981
Episode 8: reward: 17.000, steps: 972
Episode 9: reward: 17.000, steps: 969
Episode 10: reward: 17.000, steps: 957
La media del reward es: 17.0


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

In [21]:
print("La media del reward es:", np.mean(test.history["episode_reward"]))

La media del reward es: 10.4


---

In [20]:
test.history

{'episode_reward': [11.0, 20.0, 9.0, 7.0, 10.0, 5.0, 14.0, 8.0, 6.0, 14.0],
 'nb_steps': [883, 1456, 824, 829, 852, 407, 1092, 822, 837, 1200]}


### Primeros pasos

Para comenzar, ejecutamos la sesión 5 de práctica que venía por defecto en el notebook, cambiando al entorno de trabajo del proyecto una vez creamos nuestro propio entorno virtual en local. El promedio de recompensas obtenido oscilaba entre 14 y 15, lo que nos llevó a iniciar una serie de cambios para mejorar estos resultados.

### Modificaciones iniciales

Las primeras modificaciones que se nos ocurrieron fueron aumentar los pasos de entrenamiento y cambiar el learning rate, acorde a lo aprendido en la asignatura de Redes Neuronales. Aumentar los pasos de entrenamiento nos permitió proporcionar más datos y tiempo al modelo para aprender, mientras que ajustar la tasa de aprendizaje nos ayudó a controlar la velocidad a la que el modelo se ajustaba a los datos. Sin embargo, estos ajustes iniciales no fueron suficientes, por lo que decidimos hacer más compleja la arquitectura de la red neuronal. 

### Primera modificacion de la red neuronal

Empezamos añadiendo capas convolucionales, incrementando la profundidad del modelo y añadiendo normalizaciones. 

| Layer (type)                               | Output Shape       | Param # |
|--------------------------------------------|--------------------|---------|
| permute_1 (Permute)                        | (None, 84, 84, 4)  | 0       |
| conv2d_3 (Conv2D)                          | (None, 21, 21, 32) | 8224    |
| activation_5 (Activation)                  | (None, 21, 21, 32) | 0       |
| batch_normalization_4 (BatchNormalization) | (None, 21, 21, 32) | 128     |
| dropout_4 (Dropout)                        | (None, 21, 21, 32) | 0       |
| conv2d_4 (Conv2D)                          | (None, 11, 11, 64) | 32832   |
| activation_6 (Activation)                  | (None, 11, 11, 64) | 0       |
| batch_normalization_5 (BatchNormalization) | (None, 11, 11, 64) | 256     |
| dropout_5 (Dropout)                        | (None, 11, 11, 64) | 0       |
| conv2d_5 (Conv2D)                          | (None, 11, 11, 64) | 36928   |
| activation_7 (Activation)                  | (None, 11, 11, 64) | 0       |
| batch_normalization_6 (BatchNormalization) | (None, 11, 11, 64) | 256     |
| dropout_6 (Dropout)                        | (None, 11, 11, 64) | 0       |
| flatten_1 (Flatten)                        | (None, 7744)       | 0       |
| dense_2 (Dense)                            | (None, 512)        | 3965440 |
| activation_8 (Activation)                  | (None, 512)        | 0       |
| batch_normalization_7 (BatchNormalization) | (None, 512)        | 2048    |
| dropout_7 (Dropout)                        | (None, 512)        | 0       |
| dense_3 (Dense)                            | (None, 6)          | 3078    |
| activation_9 (Activation)                  | (None, 6)          | 0       |

**Total params:** 4,049,190  
**Trainable params:** 4,047,846  
**Non-trainable params:** 1,344

Expandimos la arquitectura a tres capas convolucionales, con la intención de mejorar aún más la capacidad del modelo para aprender representaciones significativas, las normalizacion se añadieron con la intencion de evitar el sobreentrenamiento y que la red aprendiera mas uniformemente.

En cuanto al agente la configuracion la mantubimos de los intentos anteriores, como no mejroraban decidimos aumentar los esteps a 1750000 pero los resultados no aumentaron mucho, la media era de 16.

### Diferentes policys

Al ver que no aumentaba mucho el reward decidimos probar con policys. Probamos con la de Boltzmann y la DecayEpsGreedy pero no obtubimos cambis significativos si no que empeoramos los resultados

### Memoria y steps_warmup


A continuación, aumentamos el parámetro del agente `steps_warmup` y el límite de la memoria de experiencia; todo esto con la intencion de guardar mas experiencias para el entrenamiento. Aumentars estos parametros hizo que se mejorara a 19 pero detectamos sobre entrenamiento y el agente siempre realizaba las mismas acciones.

### Simplificacion de la red

Para eliminar este problema se sumplifico la red para reducir los parametros entrenables


| Layer (type)                 | Output Shape         | Param #   |
|------------------------------|----------------------|-----------|
| permute (Permute)            | (None, 84, 84, 4)    | 0         |
| conv2d (Conv2D)              | (None, 20, 20, 32)   | 8224      |
| activation (Activation)      | (None, 20, 20, 32)   | 0         |
| conv2d_1 (Conv2D)            | (None, 9, 9, 64)     | 32832     |
| activation_1 (Activation)    | (None, 9, 9, 64)     | 0         |
| conv2d_2 (Conv2D)            | (None, 7, 7, 64)     | 36928     |
| activation_2 (Activation)    | (None, 7, 7, 64)     | 0         |
| flatten (Flatten)            | (None, 3136)         | 0         |
| dense (Dense)                | (None, 512)          | 1606144   |
| activation_3 (Activation)    | (None, 512)          | 0         |
| dense_1 (Dense)              | (None, 256)          | 131328    |
| activation_4 (Activation)    | (None, 256)          | 0         |
| dense_2 (Dense)              | (None, 6)            | 1542      |
| activation_5 (Activation)    | (None, 6)            | 0         |

**Total params:** 1,816,998  
**Trainable params:** 1,816,998  
**Non-trainable params:** 0

Este cambio, junto al aumento de la memoria y el warmup supuso una mejora considerable llegando a 21

### Resultados finales:

Finalmente se probo a reentrenar el modelo anterior para hacer dos fases un millon de steps cada una haciendo un total de 2 millones esto pordujo em mejor resultado 24.

| **Episode** | **Reward** | **Steps** |
|-------------|------------|-----------|
| Episode 1   | 28.000     | 1024      |
| Episode 2   | 19.000     | 590       |
| Episode 3   | 27.000     | 854       |
| Episode 4   | 22.000     | 830       |
| Episode 5   | 12.000     | 480       |
| Episode 6   | 32.000     | 1313      |
| Episode 7   | 17.000     | 604       |
| Episode 8   | 31.000     | 1157      |
| Episode 9   | 33.000     | 1089      |
| Episode 10  | 23.000     | 839       |

=======================

**Average reward: 24.4**
